<a href="https://colab.research.google.com/github/Sidhtang/text-geneator-using-lstm/blob/main/automatic_text_generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#download the data and save it to poem.txt
!wget https://www.gutenberg.org/files/1661/1661-0.txt -O book.txt

--2024-06-16 10:56:39--  https://www.gutenberg.org/files/1661/1661-0.txt
Resolving www.gutenberg.org (www.gutenberg.org)... 152.19.134.47, 2610:28:3090:3000:0:bad:cafe:47
Connecting to www.gutenberg.org (www.gutenberg.org)|152.19.134.47|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 607504 (593K) [text/plain]
Saving to: ‘book.txt’

book.txt            100%[===================>] 593.27K   549KB/s    in 1.1s    

2024-06-16 10:56:41 (549 KB/s) - ‘book.txt’ saved [607504/607504]



In [2]:
#read the file in text string
text = open('book.txt', 'r', encoding='utf-8').read()
text = text.lower()

In [3]:
#create list of sentences
sentences = text.split('\n')

In [4]:
#import dependencies to preprocess the text data and making sequences
import tensorflow as tf
import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences

In [5]:
#initialize the tokenizer, which can work char by char
tokenizer = Tokenizer(oov_token='<UNK>')

In [6]:
tokenizer.fit_on_texts(sentences)

In [7]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

8923

In [8]:
sequences=tokenizer.texts_to_sequences(sentences)

In [25]:
input_sequences = []
for sequence in sequences:
  for i in range(1, len(sequence)):
    n_gram_sequence = sequence[:i+1]
    input_sequences.append(n_gram_sequence)

In [26]:
print(input_sequences[0], input_sequences[1])

[4776, 158] [4776, 158, 331]


In [31]:
#find the maximum length among sequences
max_seq_len = max([len(seq) for seq in input_sequences])
max_seq_len

20

In [28]:
padded_sequences=pad_sequences(sequences, maxlen=max_sequence_len, )

In [29]:
print(padded_sequences[0], padded_sequences[1])

[   0    0    0    0    0    0    0    0    0    0 4776  158  331  886
    5    2 1092    5  130   35] [   0    0    0    0    0    0    0    0    0    0    0    0    0    0
    0    0   46  656 2664 2665]


In [14]:
import numpy as np
padded_sequences = np.array(padded_sequences)

In [15]:
print(len(padded_sequences[0]))
print(len(padded_sequences[1]))

20
20


In [16]:
#prepare training sequences and labels
x = padded_sequences[:, : -1]
labels = padded_sequences[:, -1]

In [17]:
#to one hot encode the labels
y = tf.keras.utils.to_categorical(labels, num_classes=vocab_size)

In [18]:
x.shape

(12307, 19)

In [19]:
y.shape

(12307, 8923)

In [20]:
#import dependencies for defining the model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Embedding, Bidirectional, LSTM, Dense
from tensorflow.keras.optimizers import Adam

In [21]:
model = Sequential()
model.add(Embedding(vocab_size, 100, input_length=max_sequence_len-1))
model.add(Bidirectional(LSTM(256)))
model.add(Dense(vocab_size, activation='softmax'))
adam=Adam(learning_rate=0.01)
model.compile(loss='categorical_crossentropy', optimizer=adam, metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding (Embedding)       (None, 19, 100)           892300    
                                                                 
 bidirectional (Bidirection  (None, 512)               731136    
 al)                                                             
                                                                 
 dense (Dense)               (None, 8923)              4577499   
                                                                 
Total params: 6200935 (23.65 MB)
Trainable params: 6200935 (23.65 MB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [22]:
#initialize the callback for early stopping the training if there is not at least 1% improvement in the accuracy
from tensorflow.keras.callbacks import EarlyStopping
es = EarlyStopping(monitor = 'acc', min_delta=0.01)


In [23]:
model.fit(x, y, epochs=50, verbose=1, batch_size=512, callbacks=[es])

Epoch 1/50
25/25 [==============================] - ETA: 0s - loss: 5.8877 - accuracy: 0.2029

25/25 [==============================] - 10s 222ms/step - loss: 5.8877 - accuracy: 0.2029
Epoch 2/50
25/25 [==============================] - ETA: 0s - loss: 4.4453 - accuracy: 0.3366

25/25 [==============================] - 3s 128ms/step - loss: 4.4453 - accuracy: 0.3366
Epoch 3/50
25/25 [==============================] - ETA: 0s - loss: 4.0247 - accuracy: 0.3727

25/25 [==============================] - 4s 148ms/step - loss: 4.0247 - accuracy: 0.3727
Epoch 4/50
25/25 [==============================] - ETA: 0s - loss: 3.7186 - accuracy: 0.3884

25/25 [==============================] - 4s 138ms/step - loss: 3.7186 - accuracy: 0.3884
Epoch 5/50
25/25 [==============================] - ETA: 0s - loss: 3.4183 - accuracy: 0.4020

...

25/25 [==============================] - 1s 35ms/step - loss: 0.0954 - accuracy: 0.9904
Epoch 47/50
25/25 [==============================] - ETA: 0s - loss: 0.0878 - accuracy: 0.9912

25/25 [==============================] - 1s 43ms/step - loss: 0.0878 - accuracy: 0.9912
Epoch 48/50
25/25 [==============================] - ETA: 0s - loss: 0.0792 - accuracy: 0.9918

25/25 [==============================] - 1s 42ms/step - loss: 0.0792 - accuracy: 0.9918
Epoch 49/50
24/25 [===========================>..] - ETA: 0s - loss: 0.0732 - accuracy: 0.9923

25/25 [==============================] - 1s 46ms/step - loss: 0.0738 - accuracy: 0.9922
Epoch 50/50
25/25 [==============================] - ETA: 0s - loss: 0.0754 - accuracy: 0.9921

25/25 [==============================] - 1s 55ms/step - loss: 0.0754 - accuracy: 0.9921


In [34]:
#Time to become storyteller!
seed_text = "I could not help laughing at the ease with which he explained his process of deduction"
next_words = 100
import numpy as np
for _ in range(next_words):
  sequence = tokenizer.texts_to_sequences([seed_text])
  padded = pad_sequences(sequence, maxlen=max_seq_len-1)
  predicted = np.argmax(model.predict(padded, verbose=0), axis=-1) # Use np.argmax to get the class with highest probability
  output_word = ''
  for word, index in tokenizer.word_index.items():
    if index == predicted:
      output_word = word
      break
  seed_text += ' ' + output_word
print(seed_text)

I could not help laughing at the ease with which he explained his process of deduction i sit down into left then i do i had a man i do not do in twenty are i am not very door ” i said he “that most are do your most “g” ” i asked my most little ” i have had to my hair ” i said i am a moment dog hair white town am in his eager and i door i did you to do ” i asked holmes to do in the matter ” i give the stream and a few john my little “g” i am a “g” “g” i do not
